In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem
import scipy.optimize as opt
import warnings
import geopy.distance
import matplotlib as mpl

from scipy.ndimage.measurements import label
from scipy.ndimage import binary_dilation

warnings.filterwarnings('ignore')

dir_data='/Users/lexihenny/'
dir2='/Volumes/Extreme Pro/'
dir3='/Volumes/Extreme Pro 1/'

In [ ]:
yrs=np.arange(1980,2024,1)

if 1==1:
    ds=xr.open_dataset(dir_data+'ar_vars_geomeans_merra2_MAX.nc')
    print(ds)
    ivt_concat_merra2_fixed_poleward=ds['ivt_concat_merra2_fixed_poleward'].values
    iwv_concat_merra2_fixed_poleward=ds['iwv_concat_merra2_fixed_poleward'].values
    s_concat_merra2_fixed_poleward=ds['s_concat_merra2_fixed_poleward'].values
    mfc_concat_merra2_fixed_poleward=ds['mfc_concat_merra2_fixed_poleward'].values
    
    ivt_concat_merra2_fixed=ds['ivt_concat_merra2_fixed'].values
    iwv_concat_merra2_fixed=ds['iwv_concat_merra2_fixed'].values
    s_concat_merra2_fixed=ds['s_concat_merra2_fixed'].values
    mfc_concat_merra2_fixed=ds['mfc_concat_merra2_fixed'].values
    
    ivt_concat_merra2_polar=ds['ivt_concat_merra2_variable'].values
    iwv_concat_merra2_polar=ds['iwv_concat_merra2_variable'].values
    s_concat_merra2_polar=ds['s_concat_merra2_variable'].values
    mfc_concat_merra2_polar=ds['mfc_concat_merra2_variable'].values
    
    
ds=xr.open_dataset(dir3+'ar_vars_geomeans_era5.nc')
lons=ds.lon.values
lats=ds.lat.values
for i in range(40,len(yrs)):
    year=yrs[i]
    print(year)
    
    ds=xr.open_dataset(dir2+'merra2.ivt.3hr.model.'+str(year)+'.nc')
    ivt=ds['ivt'][0::2,:,:]
    
    ds=xr.open_dataset(dir2+'merra2.iwv.3hr.model.'+str(year)+'.nc')
    iwv=ds['iwv'][0::2,:,:]
    
    ds=xr.open_dataset(dir3+'merra2.u850.6hr.'+str(year)+'.nc')
    u=ds['u']
    ds=xr.open_dataset(dir3+'merra2.v850.6hr.'+str(year)+'.nc')
    v=ds['v']
    s=np.sqrt(u**2+v**2)
        
    ds=xr.open_dataset(dir2+'merra2.mfc.3hr.model.'+str(year)+'.nc')#merra-2 is tendency
    mfc=ds['mfc'][0::2,:,:]
    
    ds=xr.open_dataset(dir2+'merra2.ar.labels.model.variable.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    iwv['time']=labeled.time.values
    s['time']=labeled.time.values
    
    ivt['lat']=lats
    ivt['lon']=lons
    iwv['lat']=lats
    iwv['lon']=lons
    s['lat']=lats
    s['lon']=lons
    mfc['lat']=lats
    mfc['lon']=lons
    labeled['lat']=lats
    labeled['lon']=lons
    
    iwv_sel=iwv.where(labeled>0)
    iwv_mean=iwv_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    ivt_sel=ivt.where(labeled>0)
    ivt_mean=ivt_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    s_sel=s.where(labeled>0)
    s_mean=s_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    mfc_sel=mfc.where(labeled>0)
    mfc_mean=mfc_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    
    
    if i==0:
        ivt_concat_merra2_polar=ivt_mean
        iwv_concat_merra2_polar=iwv_mean
        s_concat_merra2_polar=s_mean
        mfc_concat_merra2_polar=mfc_mean
    else:
        #ivt_concat_merra2_polar=xr.concat([ivt_concat_merra2_polar,ivt_mean],dim='time')
        #iwv_concat_merra2_polar=xr.concat([iwv_concat_merra2_polar,iwv_mean],dim='time')
        #s_concat_merra2_polar=xr.concat([s_concat_merra2_polar,s_mean],dim='time')
        #mfc_concat_merra2_polar=xr.concat([mfc_concat_merra2_polar,mfc_mean],dim='time')
        
        ivt_concat_merra2_polar=np.concatenate((ivt_concat_merra2_polar,ivt_mean),axis=0)
        iwv_concat_merra2_polar=np.concatenate((iwv_concat_merra2_polar,iwv_mean),axis=0)
        s_concat_merra2_polar=np.concatenate((s_concat_merra2_polar,s_mean),axis=0)
        mfc_concat_merra2_polar=np.concatenate((mfc_concat_merra2_polar,mfc_mean),axis=0)
    
    
    ds=xr.open_dataset(dir2+'merra2.ar.labels.model.'+str(year)+'.nc')
    labeled=ds['ar_labeled'][0::2,:,:]
    iwv['time']=labeled.time.values
    s['time']=labeled.time.values
    
    labeled['lat']=lats
    labeled['lon']=lons
    
    iwv_sel=iwv.where(labeled>0)
    iwv_mean=iwv_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    ivt_sel=ivt.where(labeled>0)
    ivt_mean=ivt_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    s_sel=s.where(labeled>0)
    s_mean=s_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    mfc_sel=mfc.where(labeled>0)
    mfc_mean=mfc_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    
    if i==0:
        ivt_concat_merra2_fixed=ivt_mean
        iwv_concat_merra2_fixed=iwv_mean
        s_concat_merra2_fixed=s_mean
        mfc_concat_merra2_fixed=mfc_mean
    else:
        #ivt_concat_merra2_fixed=xr.concat([ivt_concat_merra2_fixed,ivt_mean],dim='time')
        #iwv_concat_merra2_fixed=xr.concat([iwv_concat_merra2_fixed,iwv_mean],dim='time')
        #s_concat_merra2_fixed=xr.concat([s_concat_merra2_fixed,s_mean],dim='time')
        #mfc_concat_merra2_fixed=xr.concat([mfc_concat_merra2_fixed,mfc_mean],dim='time')
        
        ivt_concat_merra2_fixed=np.concatenate((ivt_concat_merra2_fixed,ivt_mean),axis=0)
        iwv_concat_merra2_fixed=np.concatenate((iwv_concat_merra2_fixed,iwv_mean),axis=0)
        s_concat_merra2_fixed=np.concatenate((s_concat_merra2_fixed,s_mean),axis=0)
        mfc_concat_merra2_fixed=np.concatenate((mfc_concat_merra2_fixed,mfc_mean),axis=0)
    

    ds=xr.open_dataset(dir2+'merra2.ar.labels.model.poleward.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    iwv['time']=labeled.time.values
    s['time']=labeled.time.values
    
    labeled['lat']=lats
    labeled['lon']=lons
    
    iwv_sel=iwv.where(labeled>0)
    iwv_mean=iwv_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    ivt_sel=ivt.where(labeled>0)
    ivt_mean=ivt_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    s_sel=s.where(labeled>0)
    s_mean=s_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    mfc_sel=mfc.where(labeled>0)
    mfc_mean=mfc_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    
    if i==0:
        ivt_concat_merra2_fixed_poleward=ivt_mean
        iwv_concat_merra2_fixed_poleward=iwv_mean
        s_concat_merra2_fixed_poleward=s_mean
        mfc_concat_merra2_fixed_poleward=mfc_mean
    else:
        #ivt_concat_merra2_fixed_poleward=xr.concat([ivt_concat_merra2_fixed_poleward,ivt_mean],dim='time')
        #iwv_concat_merra2_fixed_poleward=xr.concat([iwv_concat_merra2_fixed_poleward,iwv_mean],dim='time')
        #s_concat_merra2_fixed_poleward=xr.concat([s_concat_merra2_fixed_poleward,s_mean],dim='time')
        #mfc_concat_merra2_fixed_poleward=xr.concat([mfc_concat_merra2_fixed_poleward,mfc_mean],dim='time')
        
        ivt_concat_merra2_fixed_poleward=np.concatenate((ivt_concat_merra2_fixed_poleward,ivt_mean),axis=0)
        iwv_concat_merra2_fixed_poleward=np.concatenate((iwv_concat_merra2_fixed_poleward,iwv_mean),axis=0)
        s_concat_merra2_fixed_poleward=np.concatenate((s_concat_merra2_fixed_poleward,s_mean),axis=0)
        mfc_concat_merra2_fixed_poleward=np.concatenate((mfc_concat_merra2_fixed_poleward,mfc_mean),axis=0)
        
        
print('DONE')

In [ ]:
dk=xr.Dataset()
dk['ivt_concat_merra2_variable']=(('time','lat','lon'),ivt_concat_merra2_polar)
dk['iwv_concat_merra2_variable']=(('time','lat','lon'),iwv_concat_merra2_polar)
dk['s_concat_merra2_variable']=(('time','lat','lon'),s_concat_merra2_polar)
dk['mfc_concat_merra2_variable']=(('time','lat','lon'),mfc_concat_merra2_polar)

dk['ivt_concat_merra2_fixed']=(('time','lat','lon'),ivt_concat_merra2_fixed)
dk['iwv_concat_merra2_fixed']=(('time','lat','lon'),iwv_concat_merra2_fixed)
dk['s_concat_merra2_fixed']=(('time','lat','lon'),s_concat_merra2_fixed)
dk['mfc_concat_merra2_fixed']=(('time','lat','lon'),mfc_concat_merra2_fixed)

dk['ivt_concat_merra2_fixed_poleward']=(('time','lat','lon'),ivt_concat_merra2_fixed_poleward)
dk['iwv_concat_merra2_fixed_poleward']=(('time','lat','lon'),iwv_concat_merra2_fixed_poleward)
dk['s_concat_merra2_fixed_poleward']=(('time','lat','lon'),s_concat_merra2_fixed_poleward)
dk['mfc_concat_merra2_fixed_poleward']=(('time','lat','lon'),mfc_concat_merra2_fixed_poleward)

dk.coords['time']=np.arange(1980,2024)
dk.coords['lat']=ivt.lat.values
dk.coords['lon']=ivt.lon.values
dk['lat'].attrs["units"]='degrees_north'
dk['lon'].attrs["units"]='degrees_east'
dk['time'].attrs["units"]='years'
print(dk)

try:
    os.remove(dir_data+'ar_vars_geomeans_merra2_MAX.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'ar_vars_geomeans_merra2_MAX.nc',mode='w',format='NETCDF4')

In [ ]:
test=ivt_concat_merra2_fixed_poleward
fig=plt.figure(figsize=(20,10))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
ax.pcolormesh(ivt.lon,ivt.lat,test[5,:,:],vmin=0,vmax=1800,cmap=plt.cm.seismic)
ax.coastlines(resolution='10m')
ax.plot(ivt.lon[50],ivt.lat[50],'ko',markersize=12)
plt.show()

In [ ]:
if 1==1:
    ds=xr.open_dataset(dir_data+'ar_vars_geomeans_merra2_MAX.nc')
    ivt_concat_merra2_fixed_poleward=ds['ivt_concat_merra2_fixed_poleward'].values
    iwv_concat_merra2_fixed_poleward=ds['iwv_concat_merra2_fixed_poleward'].values
    s_concat_merra2_fixed_poleward=ds['s_concat_merra2_fixed_poleward'].values
    mfc_concat_merra2_fixed_poleward=ds['mfc_concat_merra2_fixed_poleward'].values
    
    ivt_concat_merra2_fixed=ds['ivt_concat_merra2_fixed'].values
    iwv_concat_merra2_fixed=ds['iwv_concat_merra2_fixed'].values
    s_concat_merra2_fixed=ds['s_concat_merra2_fixed'].values
    mfc_concat_merra2_fixed=ds['mfc_concat_merra2_fixed'].values
    
    ivt_concat_merra2_polar=ds['ivt_concat_merra2_variable'].values
    iwv_concat_merra2_polar=ds['iwv_concat_merra2_variable'].values
    s_concat_merra2_polar=ds['s_concat_merra2_variable'].values
    mfc_concat_merra2_polar=ds['mfc_concat_merra2_variable'].values